In [1]:
import ray 

ray.init(num_gpus=1)

2022-11-14 17:35:22,203	INFO worker.py:1528 -- Started a local Ray instance.


Python version:,3.8.13
Ray version:,2.1.0


In [2]:
import ray
from ray.rllib.agents.ppo import PPOTrainer
config = {
    "env": "LunarLander-v2",
    "num_gpus":1,
}
stop = {"episode_reward_mean": 250}
ray.shutdown()
ray.init(
  num_gpus=1,
  include_dashboard=False,
  ignore_reinit_error=True,
  log_to_driver=False,
)
# execute training 
analysis = ray.tune.run(
  "PPO",
  config=config,
  stop=stop,
  checkpoint_at_end=True,
  local_dir="LunarLander_v2"  
)

2022-11-14 17:35:39,067	INFO worker.py:1528 -- Started a local Ray instance.


Trial name,agent_timesteps_total,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,experiment_id,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_recreated_workers,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
PPO_LunarLander-v2_0c444_00000,332000,"{'num_env_steps_sampled': 332000, 'num_env_steps_trained': 332000, 'num_agent_steps_sampled': 332000, 'num_agent_steps_trained': 332000}",{},2022-11-14_18-01-31,True,298.81,{},308.179,251.193,1.39381,12,1135,73977bc9121149b7961235f74406c494,Davids-MacBook-Pro.local,"{'learner': {'default_policy': {'learner_stats': {'cur_kl_coeff': 0.20000000298023224, 'cur_lr': 4.999999873689376e-05, 'total_loss': 8.935843, 'policy_loss': -0.007028516, 'vf_loss': 8.942245, 'vf_explained_var': -0.31424975, 'kl': 0.0031302692, 'entropy': 0.65838414, 'entropy_coeff': 0.0, 'model': {}}, 'train': None}}, 'num_env_steps_sampled': 332000, 'num_env_steps_trained': 332000, 'num_agent_steps_sampled': 332000, 'num_agent_steps_trained': 332000}",83,127.0.0.1,332000,332000,332000,4000,332000,4000,0,2,0,4000,"{'cpu_util_percent': 38.87037037037038, 'ram_util_percent': 64.15185185185186}",9759,{},{},{},"{'mean_raw_obs_processing_ms': 0.1601326764945736, 'mean_inference_ms': 4.187314691157777, 'mean_action_processing_ms': 0.0473767497168801, 'mean_env_wait_ms': 0.43318341157847284, 'mean_env_render_ms': 0.0}","{'episode_reward_max': 308.1786249161687, 'episode_reward_min': 1.3938132851965719, 'episode_reward_mean': 251.19291254430166, 'episode_len_mean': 298.81, 'episode_media': {}, 'episodes_this_iter': 12, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [278.50992876804804, 249.13046516509831, 285.42422382345865, 308.1786249161687, 279.19237131399063, 280.6714432989439, 252.51606338424864, 209.31325550740002, 1.3938132851965719, 252.25937974465234, 295.9549401561625, 290.77807463954434, 292.0819586560402, 281.3009462672742, 255.0752924204666, 263.97278539280023, 186.8589497946021, 248.05486414637062, 255.29793412278167, 253.64009954395345, 256.2431563937039, 252.67171848313436, 273.9966250266839, 284.3045784197082, 286.62731308569505, 297.04403081669, 255.8825100633388, 263.2956310392441, 245.01501691393565, 263.9226977948465, 267.74553542616485, 239.23242430623387, 251.11326609727132, 280.45913042292113, 229.4166768042906, 81.49435898623074, 263.666366422357, 282.78240423850514, 200.23539883500717, 231.50304427600457, 150.07477331270988, 280.7978001924905, 232.27455371061265, 274.37300880936334, 267.6315795894086, 251.81233689077126, 242.79101459673686, 181.77083931701947, 189.00616237584845, 279.1468086123799, 262.9075661620287, 278.39972140264115, 262.202146953348, 231.6703700547995, 252.9142788884611, 16.108324960117542, 262.20560373142064, 233.51467612700156, 266.184355826431, 255.30266716615515, 229.1343880917675, 255.08697665752035, 276.2837105843396, 279.9340928476405, 266.7911208991543, 252.89118186231272, 293.53900192285704, 267.81294332896215, 267.11488884628886, 304.12830739004255, 246.50752990771196, 272.92177119975184, 250.52814113456662, 297.80750092328066, 276.5393538910248, 264.6550066115549, 272.30292020817535, 245.65941068383611, 266.46437465089457, 291.92529520503115, 257.4643014888552, 288.59283738229277, 264.89345583554905, 274.74201747395693, 235.09576954950364, 280.6095445638396, 261.6028755413852, 296.8684316496339, 229.31588835807278, 221.89310640136893, 243.0876252453735, 265.8436016335572

2022-11-14 18:01:31,899	INFO tune.py:777 -- Total run time: 1552.15 seconds (1551.63 seconds for the tuning loop).


In [3]:
# restore a trainer from the last checkpoint
trial = analysis.get_best_logdir("episode_reward_mean", "max")
checkpoint = analysis.get_best_checkpoint(
  trial,
  "training_iteration",
  "max",
)
trainer = PPOTrainer(config=config)
trainer.restore(checkpoint)

2022-11-14 18:01:41,607	INFO algorithm.py:2303 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2022-11-14 18:01:41,608	INFO ppo.py:379 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-11-14 18:01:41,609	INFO algorithm.py:457 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


Metal device set to: Apple M1 Pro


2022-11-14 18:01:46,902	WARNING util.py:66 -- Install gputil for GPU system monitoring.
2022-11-14 18:01:46,952	INFO trainable.py:766 -- Restored on 127.0.0.1 from checkpoint: /Users/davidkwak/tensorflow-test/gym_env/Box2D/LunarLander_v2/PPO/PPO_LunarLander-v2_0c444_00000_0_2022-11-14_17-35-39/checkpoint_000083
2022-11-14 18:01:46,953	INFO trainable.py:775 -- Current state after restoring: {'_iteration': 83, '_timesteps_total': None, '_time_total': 1540.9871110916138, '_episodes_total': 1135}


In [6]:
import gym
from gym.wrappers import RecordVideo

env = RecordVideo(gym.make("LunarLander-v2"),"ppo_video")
obs = env.reset()

while True:
    action = trainer.compute_action(obs)
    obs,reward,done, _ = env.step(action)
    if done:
        break
env.close()

/Users/davidkwak/tensorflow-test/env/lib/python3.8/site-packages/gym/wrappers/record_video.py:41: UserWarning: WARN: Overwriting existing videos at /Users/davidkwak/tensorflow-test/gym_env/Box2D/ppo_video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
E1114 18:02:18.243444000 8116724992 fork_posix.cc:76]                  Other threads are currently calling into gRPC, skipping fork() handlers
E1114 18:02:18.256518000 8116724992 fork_posix.cc:76]                  Other threads are currently calling into gRPC, skipping fork() handlers
